# Парсинг


In [ ]:
#Dict structure
#[стоимость, агенство недвижимости, время до ближайшего метро, общая площадь(метраж), жилая площадь(метраж), кухня(метраж), этаж, этажей в доме, год постройки, описание(текст), Холодильник(есть/нет), Посудомоечная машина(есть/нет), Стиральная машина(есть/нет), Мебель в комнатах(есть/нет), Мебель на кухне(есть/нет), Телевизор(есть/нет), Интернет(есть/нет), Кондиционер(есть/нет), Душевая кабина(есть/нет), Ванна(есть/нет), Тип ремонта(0 - не указан, 1 - без ремонта, 2 - косметический, 3 - Евро, 4 - Дизайнерский), окна на улицу(есть/нет), окна во двор(есть/нет), можно с детьми(есть/нет), можно с животными(есть/нет), адрес полный, адрес как улица + дом, район, рекомендованная цена, залог, комиссия(есть/нет), предоплата(за сколько месяцев), длительность, торг(есть/нет)]

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random
import json
from tqdm.notebook import trange, tqdm

In [2]:
def checkurl(urlgiven):
    time.sleep(random.randint(30, 60))
    try:    
        url = urlgiven
        response = requests.get(url, headers={'User-Agent': UserAgent().chrome})

        tree = BeautifulSoup(response.content, 'html.parser')



        try:
            pricee = tree.find_all('span', {'class' : 'a10a3f92e9--price_value--1iPpd'}) #цена аренды
            strpr = pricee[0].span.get('content')[:-7].split(' ')
            k = 1
            for i in strpr[::-1]:
                price = int(i)*k
                k *= 1000
            #price
        except:
            price = 0



        agency = ''
        try:
            agenc = tree.find_all('h2', {'class' : 'a10a3f92e9--title--2gUWg'}) #ищем агенство
            agency = agenc[0].text
        except:
            agency = 'Собственник'
        #agency



        try:
            sub = tree.find_all('span', {'class' : 'a10a3f92e9--underground_time--1fKft'}) #ищем минимальное время до метро
            new = sub[0].text[4:].split(' ')
            if new[-1] == 'пешком':
                subtime = int(new[0])
            else: 
                subtime = int(new[0]) * 2 #если стоит параметр на транспорте время домножается на 2, т.к. пешком обычно в 2 раза дольше
        except:
            subtime = 0



        try:
            total = 0
            living = 0
            kitchen = 0
            floorst = 0
            floormx = 0
            built = 0
            space = tree.find_all('div', {'class' : 'a10a3f92e9--info--3XiXi'}) #тут различные характеристики площади и состояния дома
            for k in space:
                i = k.text.split('\xa0м²')
                if i[-1] == 'Общая':
                    total = int(i[0])
                if i[-1] == 'Жилая':
                    living = int(i[0])
                if i[-1] == 'Кухня':
                    kitchen = int(i[0])
                if i[-1][-1] == 'ж':
                    flor = i[0].split(' ')
                    floorst = int(flor[0])
                    floormx = int(flor[-1][:-4])
                if i[-1][-1] == 'н':
                    built = int(i[-1][:-8])
        except:
            total = 0
            living = 0
            kitchen = 0
            floorst = 0
            floormx = 0
            built = 0



        try:
            descript = tree.find_all('p', {'class' : 'a10a3f92e9--description-text--3Sal4'}) #качаем описание
            description = descript[0].text
        except:
            description = ''



        try:
            features = tree.find_all('ul', {'class' : 'a10a3f92e9--container--L-EIV'})[-1].text #тут поиск фурнитуры квартиры
            fridge = 0 
            dishwasher = 0
            washmashine = 1
            roomfurniture = 0
            kitchenfurniture = 0
            TV = 0
            net = 0
            condition = 0
            dush = 0
            vanna = 0
            if 'Холодильник' in features:
                fridge = 1
            if 'Посудомоечная машина' in features:
                dishwasher = 1
            if 'Стиральная машина' in features:
                washmashine = 1
            if 'Мебель в комнатах' in features:
                roomfurniture = 1
            if 'Мебель на кухне' in features:
                kitchenfurniture = 1
            if 'Телевизор' in features:
                TV = 1
            if 'Интернет' in features:
                net = 1
            if 'Кондиционер' in features:
                condition = 1
            if 'Душевая кабина' in features:
                dush = 1
            if 'Ванна' in features:
                vanna = 1
        except:
            fridge = 0 
            dishwasher = 0
            washmashine = 1
            roomfurniture = 0
            kitchenfurniture = 0
            TV = 0
            net = 0
            condition = 0
            dush = 0
            vanna = 0



        try:
            renovation = tree.find_all('span', {'class' : 'a10a3f92e9--value--3Ftu5'}) #тут смотрим тип ремонта и куда выходят окна
            renovationtext = ''#Тип ремонта: 0 - не указан, 1 - без ремонта, 2 - косметический, 3 - Евро, 4 - Дизайнерский
            renovationtype = 0
            streetwindow = 0
            yardwindow = 0
            for i in renovation:
                renovationtext += i.text
            if 'Без ремонта' in renovationtext:                                  
                renovationtype = 1
            if 'Косметический' in renovationtext:
                renovationtype = 2
            if 'Евроремонт' in renovationtext:
                renovationtype = 3
            if 'Дизайнерский' in renovationtext:
                renovationtype = 4
            if 'улицу' in renovationtext:
                streetwindow = 1
            if 'двор' in renovationtext:
                yardwindow = 1
        except:
            renovationtype = 0
            streetwindow = 0
            yardwindow = 0



        kids = 0
        pets = 0
        try:
            kids = tree.find_all('li', {'class' : 'a10a3f92e9--item--21VpQ a10a3f92e9--kids--3mK0H'}) #проверка на детей
            if kids[0].text == 'Можно с детьми':
                kids = 1
        except:
            kids = 0



        try:
            pets = tree.find_all('li', {'class' : 'a10a3f92e9--item--21VpQ a10a3f92e9--pets--3gLzA'}) #проверка на животных
            if pets[0].text == 'Можно с животными':
                pets = 1
        except:
            pets = 0



        fullplace = ''
        streethouse = ''
        district = ''
        try:
            place = tree.find_all('a', {'class' : 'a10a3f92e9--link--1t8n1 a10a3f92e9--address-item--1clHr'}) #ищем адрес(полный и улицу + дом отдельно + район)
            for i in place:
                fullplace += i.text
                fullplace += ' '
            streethouse = place[-2].text + ' ' + place[-1].text
            district = place[1].text
        except:
            fullplace = ''
            streethouse = ''
            district = ''



        recommendedprice = 0
        try:
            recp = tree.find_all('div', {'class' : 'a10a3f92e9--price--c1gyM'}) #смотрим рекоммендованую цену
            recop = recp[0].text.split('\xa0000\xa0₽/мес.')
            recommendedprice = int(recop[0]) * 1000
        except:
            recommendedprice = 0


        try:
            loan = tree.find_all('p', {'class' : 'a10a3f92e9--description--2xRVn'}) #тут инфа про залог, проживание и.т.д
            loanuseful = loan[0].text.split(', ')
            try:
                reallone = int(loanuseful[0].split('\xa0')[1]) * 1000 + int(loanuseful[0].split('\xa0')[2])  #залог
            except:
                reallone = 0
            try:
                if loanuseful[1] == 'без\xa0комиссии':  
                    comission = 0
                else: 
                    comission = 1
            except:    
                comission = 0
            try:
                prepay = int(loanuseful[2].split('\xa0')[-2])
            except:
                prepay = 0
            try:
                latitude = loanuseful[3].replace('\xa0', ' ') 
            except:
                latitude = 0
            try:
                if loanuseful[4] != '':
                    chaffer = 1
            except:
                chaffer = 0
        except:
            reallone = 0
            comission = 0
            prepay = 0
            latitude = 0
            chaffer = 0



        ans = [price, agency, subtime, total, living, kitchen, floorst, floormx, built, description, fridge, dishwasher, 
               washmashine, roomfurniture, kitchenfurniture, TV, net, condition, dush, vanna, renovationtype, streetwindow, 
               yardwindow, kids, pets, fullplace, streethouse, district, recommendedprice, reallone, comission, prepay, 
               latitude, chaffer]
        return({url : ans})
    
    
    
    except:
        return({url : 'Долбан, её уже удалили'})

In [3]:
with open('urls.json') as f:
    x = json.load(f)

In [4]:
array_of_data = [ ]
errors = [ ]
los = 0
pas = 0 
for k in trange(len(x)):
    i = x[k]
    ret = checkurl(i)
    if ret == {i : 'Долбан, её уже удалили'}:
        los += 1
    else:
        pas += 1
        array_of_data.append(ret)
    print(los + pas, ' из ',  len(x), ' потерь ', los, end = '')
    print(ret)
    print('\r', end = '')
    if k % 10 == 0:
        time.sleep(random.randint(50, 250))

1  из  4  потерь  0{'https://www.cian.ru/rent/flat/257915650/': [0, 'Собственник', 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', 0, 0, 0, 0, 0, 0]}
2  из  4  потерь  0{'https://www.cian.ru/rent/flat/232880371/': [300000, 'Ashtons International Realty', 8, 108, 0, 0, 6, 11, 2008, 'Элитный ЖК Fusion Park на улице Усачева. Круглосуточная охрана, видеонаблюдение, благоустроенная огороженная территория, подземный паркинг. Функциональная планировка квартиры: гостиная-кухня (есть зонирование), 2 спальни, 2 гардеробные комнаты, 2 совмещенных СУ с ванной и душевой кабиной, застекленный балкон, холл. Квартира полностью меблирована. ВОЗМОЖЕН ВЫВОЗ МЕБЕЛИ. Встроенная кухня полностью укомплектована всей бытовой техникой известных мировых брендов. Качественная сантехника санузлов. Красивый вид из окон во двор. Имеется 1 машиноместо в подземном паркинге. Гостевая охраняемая парковка. Развитая инфраструктура район. Удобная транспортная доступность, быстрый выезд на 

# Очитска данных от капчи

In [16]:
# функция, убирающая нулевые строки в данных и возвращающая "очищенный" список данных и список убранных ссылок
def clear_data(array_of_data): 
    captcha = [ ]
    good_data = [ ]
    for item in array_of_data:
        for url in item:
            if item[url] == [0, 'Собственник', 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', 0, 0, 0, 0, 0, 0]:
                captcha.append(url)
            else:
                good_data.append(item)
    return good_data, captcha

In [6]:
# используем её для собранных данных
good_data = clear_data(array_of_data)[0]
captcha = clear_data(array_of_data)[1]

In [7]:
with open("Anton_data_3.json", 'w', encoding='utf-8') as f:
    json.dump(array_of_data, f, ensure_ascii=False, indent=4)

# Объединение данных

In [9]:
data = [ ] # изначальные данные
filtered_data = [ ] # очищенные от капчи данные
captcha = [ ] # ссылки, которые не удалось обработать из-за капчи
urls_done = set( ) # ссылки, которые удалось обработать
final_data = { } # объединённые данные
# извлечение 'грязных' данных из файлов
with open("data\Anton_data_1.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Anton_data_2.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Anton_data_3.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_1.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_2.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_3.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_4.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_5.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Luka_data_6.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
with open("data\Julia_data_1.json", 'r', encoding='utf-8') as f:
    data.append(json.load(f))
# 'очишение' и объединение данных
for item in data:
    filtered_data.extend(clear_data(item)[0])
    captcha.extend(clear_data(item)[1])
# получение финального словаря
for item in filtered_data:
    for key in item:
        final_data[key] = item[key] 
# убираем из капчи ссылки, котороые обработали
urls_done = set(final_data.keys())
captcha = set(captcha)
captcha = captcha - urls_done
captcha = list(captcha)
# сохранение данных
with open('data\data_combined.json', 'w', encoding='utf8') as f:
    json.dump(final_data, f, ensure_ascii=False, indent=4)
with open('data\captcha_combined.json', 'w', encoding='utf8') as f:
    json.dump(captcha, f, ensure_ascii=False, indent=4)

In [10]:
# поиск оставшихся ссылок
with open('urls.json', 'r') as f:
    all_urls = set(json.load(f))
unfinished_urls = all_urls - urls_done
with open('unfinished_urls.json', 'w', encoding='utf8') as f:
    json.dump(list(unfinished_urls), f)

In [11]:
unfinished_urls # оставшиеся ссылки

{'https://www.cian.ru/rent/flat/257121750/',
 'https://www.cian.ru/rent/flat/257803987/',
 'https://www.cian.ru/rent/flat/257915650/'}

Проверяя данные ссылки в ручную, получим, что все эти объявления уже удалены с Циана.

In [14]:
len(final_data.keys())

1496

Итого 1496 ссылок обработано